## 스마트 포인터


- & 연산자로 이미 존재하는 데이터의 참조를 생성하는 기능과 더불어, Rust에서는 smart pointer라 불리는 참조 같은 struct를 생성하는 기능을 제공합니다.

- 고수준에서 보자면 참조는 다른 자료형에 대한 접근을 제공하는 자료형이라고 볼 수 있습니다. 
- smart pointer가 일반적인 참조와 다른 점은, 프로그래머가 작성하는 내부 로직에 기반해 작동한다는 것입니다. 여러분(프로그래머)이 바로 smart한 부분을 담당하는 겁니다.

- 일반적으로 smart pointer는 struct가 *와 . 연산자로 역참조될 때 무슨 일이 발생할지 지정하기 위해 Deref, DerefMut, 그리고 Drop trait을 구현합니다.

In [2]:
use std::ops::Deref;

struct TattleTell<T> {
    value: T,
}


impl<T> Deref for TattleTell<T> {
    type Target = T;
    fn deref(&self) -> &T {
        println!("{} was used!", std::any::type_name::<T>());
        &self.value
    }
}

In [3]:
fn main() {
    let foo = TattleTell {
        value: "secret message",
    };
    // foo가 `len` 함수를 위해 자동참조된 후
    // 여기서 역참조가 즉시 일어납니다
    println!("{}", foo.len());
}


In [4]:
main();

&str was used!
14


## 위험한 스마트 코드
smart pointer는 unsafe한 코드를 꽤 자주 쓰는 경향이 있습니다. 앞서 말했듯이, smart pointer는 Rust에서 가장 저수준의 메모리를 다루기 위한 일반적인 도구입니다.

뭐가 unsafe한 코드일까요? unsafe한 코드는 Rust 컴파일러가 보증할 수 없는 몇 가지 기능이 있다는 예외사항을 제외하고는 일반적인 코드와 완전히 똑같이 동작합니다.

unsafe한 코드의 주기능은 raw pointer를 역참조하는 것입니다. 이는 raw pointer를 메모리 상의 위치에 가져다 놓고 "데이터 구조가 여깄다!"고 선언한 뒤 사용할 수 있는 데이터 표현으로 변환하는 것을 의미합니다 (i.e. *const u8을 u8로). Rust에는 메모리에 쓰여지는 모든 바이트의 의미를 추적하는 방법은 없습니다. Rust는 raw pointer로 쓰이는 임의의 숫자에 무엇이 존재하는지 보증할 수 없기 때문에, 역참조를 unsafe { ... } 블록 안에 넣습니다.

smart pointer는 raw pointer를 역참조하는데 널리 쓰이지만, 그 기능은 잘 입증 되었습니다.

In [4]:
fn main() {
    let a: [u8; 4] = [86, 14, 73, 64];
    // 이게 원시 pointer입니다.
    // 무언가의 메모리 주소를 숫자로 가져오는 것은 완전히 안전한 일입니다
    let pointer_a = &a as *const u8 as usize;
    println!("데이터 메모리 주소: {}", pointer_a);
    // 숫자를 원시 pointer로, 다시 f32로 변환하는 것 역시
    // 안전한 일입니다.
    let pointer_b = pointer_a as *const f32;
    let b = unsafe {
        // 이건 unsafe한데,
        // 컴파일러에게 우리의 pointer가 유효한 f32라고 가정하고
        // 그 값을 변수 b로 역참조 하라고 하고 있기 때문입니다.
        // Rust는 이런 가정이 참인지 검증할 방법이 없습니다.
        *pointer_b
    };
    println!("맹세하건대 이건 파이다! {}", b);
}


In [5]:
main();

데이터 메모리 주소: 6125284044
맹세하건대 이건 파이다! 3.1415


## 스마트 포인터

이미 본 적 있는 Vec<T>나 String 같은 smart pointer를 생각해 봅시다.

Vec<T>는 바이트들의 메모리 영역을 소유하는 smart pointer입니다. Rust 컴파일러는 이 바이트들에 뭐가 존재하는지 모릅니다. smart pointer는 관리하는 메모리 영역에서 내용물을 꺼내기 위해 자기가 뭘 의미하는지 해석하고, 데이터 구조가 그 바이트들 내 어디에서 시작하고 끝나는지 추적하며, 마지막으로 raw pointer를 데이터 구조로, 또 쓰기 편한 멋지고 깔끔한 인터페이스로 역참조 합니다. (e.g. my_vec[3])

유사하게, String은 바이트들의 메모리 영역을 추적하며, 쓰여지는 내용물이 언제나 유효한 utf-8이도록 프로그램적으로 제한하며, 그 메모리 영역을 &str 자료형으로 역참조할 수 있도록 도와줍니다.

이 데이터 구조들 둘 다, 자기 할 일을 하기 위해 raw pointer에 대한 unsafe한 역참조를 사용합니다.

메모리 상세:

Rust는 C의 malloc에 상응하는 alloc과 관리할 메모리 영역을 가져오기 위한 Layout을 갖고 있습니다.

In [6]:
use std::alloc::{alloc, Layout};
use std::ops::Deref;

struct Pie {
    secret_recipe: usize,
}

impl Pie {
    fn new() -> Self {
        // 4 바이트를 요청해 봅시다
        let layout = Layout::from_size_align(4, 1).unwrap();

        unsafe {
            // 메모리 위치를 숫자로 할당하고 저장합니다
            let ptr = alloc(layout) as *mut u8;
            // pointer 연산을 사용해 u8 값 몇 개를 메모리에 써봅시다
            ptr.write(86);
            ptr.add(1).write(14);
            ptr.add(2).write(73);
            ptr.add(3).write(64);

            Pie {
                secret_recipe: ptr as usize,
            }
        }
    }
}
impl Deref for Pie {
    type Target = f32;
    fn deref(&self) -> &f32 {
        // secret_recipe pointer를 f32 raw pointer로 변환합니다
        let pointer = self.secret_recipe as *const f32;
        // 역참조 하여 &f32 값으로 리턴합니다
        unsafe { &*pointer }
    }
}
fn main() {
    let p = Pie::new();
    // Pie struct의 smart pointer를 역참조 하여
    // "파이를 만듭니다"
    println!("{:?}", *p);
}


In [7]:
main();

3.1415


## 힙에 할당된 메모리
Box는 데이터를 stack에서 heap으로 옮길 수 있게 해주는 smart pointer입니다.

이를 역참조하면 마치 원래 자료형이었던 것처럼 heap에 할당된 데이터를 편하게 쓸 수 있습니다.

In [8]:
struct Pie;

impl Pie {
    fn eat(&self) {
        println!("heap에 있으니 더 맛있습니다!")
    }
}

fn main() {
    let heap_pie = Box::new(Pie);
    heap_pie.eat();
}


In [9]:
main();

heap에 있으니 더 맛있습니다!


## 실패할 수 있는 메인 다시 보기
Rust 코드에는 많고도 많은 오류 표현 방법이 있지만, 그 중에도 standard library에는 오류를 설명하기 위한 범용 trait인 std::error::Error가 있습니다.

smart pointer인 Box를 사용하면 Box<dyn std::error::Error>를 오류 리턴 시 공통된 자료형으로 사용할 수 있는데, 이는 오류를 heap에 전파하고 특정한 자료형을 몰라도 고수준에서 상호작용할 수 있도록 해주기 때문입니다.

Tour of Rust 초반에 main은 오류를 리턴할 수 있다고 배웠습니다. 이제 우리는 오류의 데이터 구조가 Rust의 일반적인 Error trait을 구현하는 한, 프로그램에서 발생할 수 있는 거의 모든 종류의 오류를 설명할 수 있는 자료형을 리턴할 수 있습니다.

fn main() -> Result<(), Box<dyn std::error:Error>>

In [10]:
use core::fmt::Display;
use std::error::Error;

struct Pie;

#[derive(Debug)]
struct NotFreshError;

impl Display for NotFreshError {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        write!(f, "이 파이는 신선하지 않군요!")
    }
}

impl Error for NotFreshError {}

impl Pie {
    fn eat(&self) -> Result<(), Box<dyn Error>> {
        Err(Box::new(NotFreshError))
    }
}

fn main() -> Result<(), Box<dyn Error>> {
    let heap_pie = Box::new(Pie);
    heap_pie.eat()?;
    Ok(())
}


In [11]:
main();